In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Heart_rate"
cohort = "GSE12385"

# Input paths
in_trait_dir = "../../input/GEO/Heart_rate"
in_cohort_dir = "../../input/GEO/Heart_rate/GSE12385"

# Output paths
out_data_file = "../../output/preprocess/Heart_rate/GSE12385.csv"
out_gene_data_file = "../../output/preprocess/Heart_rate/gene_data/GSE12385.csv"
out_clinical_data_file = "../../output/preprocess/Heart_rate/clinical_data/GSE12385.csv"
json_path = "../../output/preprocess/Heart_rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression changes in Peripheral Blood Mononuclear cells (PBMC) induced by physical activity"
!Series_summary	"Gene expression changes in Peripheral Blood Mononuclear cells (PBMC) induced by physical activity was investigated in  sedentary middle-aged men (mean age 52.6 years and BMI 29.1) who undertook a 24-week physical activity programme with blood sampling in the pre-exercise period ,  at the end of 24-weeks prescribed physical activity , and following a two-week detraining period."
!Series_overall_design	"AgilentTM Whole Human Genome Oligo Microarrays were utilised to examine the effects of physical activity on mRNA expression profiles of the Peripheral Blood Mononuclear cells (PBMC) at 3 time points (pre-exercise, after 24 weeks physical activity, and at 26 weeks after 2 weeks detraining. There were 12 participants in this programme."
Sample Characteristics Dictionary:
{0: ['Gender: Male'], 1: ['Age (years): 48', 'Age (years): 54', 'Age

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain mRNA expression profiles using AgilentTM Whole Human Genome Oligo Microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Heart rate is not explicitly mentioned in the sample characteristics
# However, there is VO2 max data which is related to cardiorespiratory fitness
# VO2 max is closely associated with heart rate but it's not heart rate itself
# Since we don't have direct heart rate measurements, we need to set trait_row to None
trait_row = None  # Heart rate not available

# Age is available in row 1
age_row = 1

# Gender is available in row 0 but all subjects are male (constant feature)
# Since it's a constant feature, we'll set it to None
gender_row = None  # Only one gender (male) present

# 2.2 Data Type Conversion
def convert_trait(value):
    # This won't be used since trait data is not available
    # But defining it for completeness
    if pd.isna(value) or value is None:
        return None
    
    parts = value.split(': ')
    if len(parts) > 1:
        try:
            return float(parts[1])  # Assuming heart rate would be continuous
        except (ValueError, TypeError):
            return None
    return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    
    parts = value.split(': ')
    if len(parts) > 1:
        try:
            return float(parts[1])  # Age as continuous variable
        except (ValueError, TypeError):
            return None
    return None

def convert_gender(value):
    # This won't be used but defining it for completeness
    if pd.isna(value) or value is None:
        return None
    
    parts = value.split(': ')
    if len(parts) > 1:
        gender = parts[1].strip().lower()
        if gender == 'male':
            return 1
        elif gender == 'female':
            return 0
    return None

# 3. Save Metadata
# Determine trait data availability (trait_row is None, so it's not available)
is_trait_available = trait_row is not None

# Initial filtering and saving cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip the clinical feature extraction step
# (This step would be executed if trait_row were not None)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 73
Header line: "ID_REF"	"GSM310611"	"GSM310633"	"GSM310636"	"GSM310637"	"GSM310638"	"GSM310639"	"GSM310736"	"GSM310737"	"GSM310738"	"GSM310739"	"GSM310744"	"GSM310745"	"GSM310746"	"GSM310747"	"GSM310748"	"GSM310749"	"GSM310750"	"GSM310751"	"GSM310752"	"GSM310753"	"GSM310754"	"GSM310755"	"GSM310756"	"GSM310757"	"GSM310758"	"GSM310759"	"GSM310760"	"GSM310761"	"GSM310763"	"GSM310765"	"GSM310768"	"GSM310770"	"GSM310774"	"GSM310775"	"GSM310776"	"GSM310777"
First data line: 1	1.33E+05	1.87E+05	1.72E+05	1.22E+05	1.20E+05	1.61E+05	1.40E+05	9.68E+04	1.47E+05	1.56E+05	1.52E+05	1.46E+05	1.77E+05	2.02E+05	2.12E+05	1.82E+05	1.62E+05	1.61E+05	2.01E+05	1.04E+05	2.03E+05	2.40E+05	2.34E+05	2.01E+05	1.58E+05	1.95E+05	1.59E+05	1.58E+05	1.48E+05	1.74E+05	1.59E+05	1.48E+05	2.05E+05	1.62E+05	1.67E+05	1.61E+05
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers from the gene expression data
# The identifiers in the first line appear to be numeric (1, 2, 3, etc.)
# These are not standard human gene symbols, which would typically be alphanumeric like BRCA1, TP53, etc.
# These are likely probe IDs or other platform-specific identifiers that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE12385
Line 6: !Series_title = Gene expression changes in Peripheral Blood Mononuclear cells (PBMC) induced by physical activity
Line 7: !Series_geo_accession = GSE12385
Line 8: !Series_status = Public on Aug 03 2010
Line 9: !Series_submission_date = Aug 08 2008
Line 10: !Series_last_update_date = Feb 22 2018
Line 11: !Series_pubmed_id = 20368384
Line 12: !Series_summary = Gene expression changes in Peripheral Blood Mononuclear cells (PBMC) induced by physical activity was investigated in  sedentary middle-aged men (mean age 52.6 years and BMI 29.1) who undertook a 24-week physical activity programme with blood sampling in the pre-exercise period ,  at the end of 24-weeks prescribed physical activity , an


Gene annotation preview:
{'ID': [1, 2, 3, 4, 5], 'COL': [266, 266, 266, 266, 266], 'ROW': [170, 168, 166, 164, 162], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1, 2, 3, 4, 5]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation data
# From the preview, 'ID' contains the probe identifiers seen in gene_data.index (numeric values)
# 'GENE_SYMBOL' contains the gene symbols we want to map to

# 2. Extract the mapping columns from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply the gene mapping to convert probe IDs to gene symbols
# This divides expression values for probes that map to multiple genes
# and sums contributions from all probes mapping to the same gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print summary of the gene data after mapping
print(f"Number of genes after mapping: {len(gene_data)}")
print("First 5 gene symbols:")
print(gene_data.index[:5])


Number of genes after mapping: 18379
First 5 gene symbols:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Apply normalization to the gene data from Step 6
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Genes before normalization: {len(gene_data)}, after: {len(normalized_gene_data)}")

# Save the normalized gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Define convert functions for trait, age, and gender
def convert_age(value):
    """Convert age value to continuous numeric type."""
    if pd.isna(value):
        return None
    try:
        # Extract the numerical value after the colon
        parts = value.split(":")
        if len(parts) < 2:
            return None
        return float(parts[1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if pd.isna(value):
        return None
    try:
        gender = value.split(":")[1].strip().lower()
        if "male" in gender:
            return 1
        elif "female" in gender:
            return 0
        else:
            return None
    except (ValueError, IndexError):
        return None

# 3. Load the clinical data again for clarity
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# From step 2, we determined that trait data is not available and gender is constant
trait_row = None  # Heart rate data is not directly available
age_row = 1       # Age data is available in row 1
gender_row = None  # Only one gender (male) present, so it's a constant feature

# Create a minimal DataFrame with sample IDs for validation
# Use the columns from the gene expression data as samples
sample_ids = normalized_gene_data.columns
minimal_df = pd.DataFrame(index=sample_ids)

# Since trait data is not available, we mark the dataset as biased (unusable for this trait)
is_biased = True  # Dataset is unusable for Heart_rate trait since it's not available

# 4. Final validation and save
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = trait_row is not None
note = "Dataset contains gene expression data from male participants undergoing physical activity program. No direct heart rate measurements available, only VO2 max which is related to cardiorespiratory fitness."

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,  # Dataset is unusable due to missing trait
    df=minimal_df,  # Minimal DataFrame with sample IDs
    note=note
)

# 5. Since trait data is not available (as determined in step 2), 
# we can't create useful linked data, so we skip saving it
print("Trait data is not available. No linked data was created or saved.")

Genes before normalization: 18379, after: 17901

Normalized gene data saved to ../../output/preprocess/Heart_rate/gene_data/GSE12385.csv
Abnormality detected in the cohort: GSE12385. Preprocessing failed.
Trait data is not available. No linked data was created or saved.
